In [14]:
import cv2
from sklearn.cluster import KMeans
import numpy as np

def transform_matrix(matrix, p, vid_shape, gt_shape):
    p = (p[0]*1280/vid_shape[1], p[1]*720/vid_shape[0])
    px = (matrix[0][0]*p[0] + matrix[0][1]*p[1] + matrix[0][2]) / ((matrix[2][0]*p[0] + matrix[2][1]*p[1] + matrix[2][2]))
    py = (matrix[1][0]*p[0] + matrix[1][1]*p[1] + matrix[1][2]) / ((matrix[2][0]*p[0] + matrix[2][1]*p[1] + matrix[2][2]))
    print(px)
    p_after = (int(px*gt_shape[1]/115) , int(py*gt_shape[0]/74))

    return p_after


In [58]:
import cv2
import numpy as np

# Load the image of the football pitch with four corners
img_football_pitch = cv2.imread('/Users/menghang/Desktop/Screenshot 2023-08-03 at 12.20.57 PM.png')

# Define the four corners of the football pitch image
# Replace these values with the actual coordinates of the four corners in your image
corners_img = np.array([[         
        572.5,
          43.5
        ],
        [
          1435.0,
          43.5
        ],
        [
          1931.875,
          440.375
        ],
        [
          60.0,
          449.75
        ]], dtype=np.float32)

# Load the template football pitch image in 2D
img_template = cv2.imread('/Users/menghang/Desktop/ml-dev/football-server/tracker/inference/black.jpg')

print(img_template.shape)
# Define the four corners of the template football pitch
# Replace these values with the actual coordinates of the four corners in your template
corners_template = np.array([[
          26.29411764705882,
          21.352941176470587
        ],
        [
          1893.941176470588,
          21.352941176470587
        ],
        [
          1899.8235294117646,
          1059.5882352941176
        ],
        [
          21.882352941176467,
          1058.1176470588234
        ]], dtype=np.float32)

# Calculate the homography matrix using the four corners
M, _ = cv2.findHomography(corners_img, corners_template)

# Now, let's assume you have detected the positions of the players in the original image
# Let's say the positions of the players are stored in a list called 'player_positions'
# Each element in the list is a tuple containing the (x, y) coordinates of a player
# Replace this with your actual list of detected player positions
player_positions = [[430 , 463]]

# Create an array of shape (N, 1, 2) from the player_positions list
# where N is the number of players detected
player_positions_array = np.array(player_positions, dtype=np.float32).reshape(-1, 1, 2)
# Use the homography matrix to transform the player positions to the 2D template
player_positions_2d = cv2.perspectiveTransform(player_positions_array, M)

# Now, player_positions_2d contains the transformed positions of the players in the 2D template
# You can use these positions to draw circles or markers on the template image to visualize the players

# Display the transformed positions on the 2D template
for player_position in player_positions_2d:
    x, y = player_position[0]
   
    cv2.circle(img_template, (int(x),int(y)), 5, (0, 255, 0), -1)

# Display the original image with the detected player positions
cv2.imshow('Original Football Pitch', img_football_pitch)

# Display the 2D template image with the transformed player positions
cv2.imshow('2D Template', img_template)

cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.waitKey(1)

(1080, 1920, 3)


-1

: 